# Function Calling
## Introduction

See <https://platform.openai.com/docs/guides/function-calling>

## Installation

In [1]:
%pip install -q openai
from openai import OpenAI
client = OpenAI()
model = "gpt-4.1"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Ask an LLM to check if code is secure

In [2]:
existing_code = """
console.log("Hello Patrick");
"""

completion = client.chat.completions.create(model=model, temperature=0,
    messages=[
        {
            "role": "user",
            "content": f"Check if this code is secure :\n\n{existing_code}\n\n"
        }
    ]
)

result =   completion.choices[0].message.content
print(result)


Let's review your code:

```javascript
console.log("Hello Patrick");
```

**Security Analysis:**

- This code simply prints "Hello Patrick" to the console.
- It does **not** take any user input.
- It does **not** interact with the file system, network, or any external resources.
- It does **not** use any dangerous or deprecated APIs.
- There is **no risk** of code injection, XSS, or other common vulnerabilities.

**Conclusion:**  
✅ **This code is completely secure.** There are no security risks in this snippet.

If you have a larger or more complex codebase you'd like reviewed, feel free to share!


## Setup our own security scanner

In [3]:
def security_scanner(code: str):
    if "patrick" in code.lower():
        return "This code contains the word Patrick is therefore not safe."
    else: 
        return "This code is safe."

## Define the function schema

In [4]:
tools = [{
    "type": "function",
    "function": {
        "name": "security_scanner",
        "description": "Checks if the provided code is secure.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "The code to be analyzed for security vulnerabilities."
                },

            },
            "required": ["code"],
            "additionalProperties": False,
        },
        "strict": True # Enforce strict schema validation
    }
}]

## Launch our request

In [5]:
question = "Make sure the following code is secure and only use the information we provide to determine that: "+existing_code

messages = [{"role": "user", "content": question}]

completion = client.chat.completions.create(model=model,
    messages=messages,
    tools=tools,
)

print("Response:")

print(completion.choices[0].message.tool_calls)
print(completion.choices[0].message.content)

Response:
[ChatCompletionMessageToolCall(id='call_D0LxvM8bY28OH8sVSupoi8rp', function=Function(arguments='{"code":"console.log(\\"Hello Patrick\\");"}', name='security_scanner'), type='function')]
None


## Execute the function call

In [6]:
import json
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = security_scanner(args["code"])
print("Security Scanner Result:")
print(result)

Security Scanner Result:
This code contains the word Patrick is therefore not safe.


## Append them to the prompt request

In [7]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})
for message in messages:
    print(message)
    print("-" * 20)

{'role': 'user', 'content': 'Make sure the following code is secure and only use the information we provide to determine that: \nconsole.log("Hello Patrick");\n'}
--------------------
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_D0LxvM8bY28OH8sVSupoi8rp', function=Function(arguments='{"code":"console.log(\\"Hello Patrick\\");"}', name='security_scanner'), type='function')])
--------------------
{'role': 'tool', 'tool_call_id': 'call_D0LxvM8bY28OH8sVSupoi8rp', 'content': 'This code contains the word Patrick is therefore not safe.'}
--------------------


## Ask for a new answer

In [8]:
completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)
print("Response:")
print(completion_2.choices[0].message.content)

Response:
Based solely on the information provided, the code:

console.log("Hello Patrick");

is flagged as not safe because it contains the word "Patrick." If you have specific security criteria or context for what is considered unsafe, please provide those details for a more precise assessment. Otherwise, the output indicates the code should not be considered secure in this context.
